<h3>Python - SQL ORM - сценарий для TikTok @_rioran_</h3>

Python - управление базой данных в стиле ORM!

Где ORM - это object-relational mapping. Или по-русски "Объектно-реляционное отображение". Ну или совсем по-русски - вместо SQL кода работаем с классами и объектами.

Код из этого ролика ищите на моём гитхабе, ссылка внутри профиля.

Смотреть будем на примере sql алхимии, но упомяну что peewee тоже достойный инструмент. А если работаете с Django - у него своя встроенная ORM.

Чтобы общаться с базой данных - нам нужен движок, подтянем его из основной библиотеки. Создадим движок под базу sqlite, дадим базе какое-нибудь мутное имя, пусть будет "мои события".

Ещё, если интересно видеть каждый SQL-запрос, который под капотом будет отправлять движок - можете включить параметр echo. По умолчанию он выключен.

In [ ]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///my_events.db')  # , echo=True)

Чтобы с помощью движка строить объекты в базе - из orm библиотеки подтянем класс declarative_base и поднимем его инстанс с именем Base. 

In [ ]:
from sqlalchemy.orm import declarative_base

Base = declarative_base()

Всё, мы готовы поднимать базу и таблицы. Из основной библиотеки возьмём инструмент для объявления столбца таблицы и всякие типы данных.

А из sql-ной части библиотеки - func для работы со встроенными функциями базы.

In [ ]:
from sqlalchemy import create_engine, \
    Column, Integer, String, DateTime

from sqlalchemy.sql import func

Предположим, мы хотим отслеживать логи событий, например. Для создания таблицы в базе объявим класс Event_Log.

Дадим таблице имя, пусть будет events. Накидаем столбцы, хочу id как уникальный числовой ключ каждого события, текстовое имя и дату-время (datetime), чтобы заполнялось в момент создания по серверному времени.

Волшебный метод repr не обязателен, но через него нам будет проще заглянуть в душу каждой строке из базы.

Ну и наконец, чтобы файл базы появился на нашем диске, если его нет - через метаданные и движок всё создаём.

In [ ]:
class Event_Log(Base):
    __tablename__ = 'events'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    dttm = Column(
        DateTime(timezone=True), 
        server_default=func.now()
    )

    def __repr__(self):
        return f'{self.id}: {self.name}'
    
Base.metadata.create_all(engine)

Самый ходовой способ работы с базой - через механизм сессий, из orm-ки подтянем класс для их создания. И привяжемся к движку.

In [ ]:
from sqlalchemy.orm import declarative_base, sessionmaker

Session = sessionmaker(bind=engine)

Ну вот и всё, подготовительные шаги закончились, мы можем наполнять нашу базу данными. Давайте через сессию закинем несколько ивентов. Работаю через with - он же наше окно в контекстный менеджер - для автоматического закрытия сессии.

После добавления в базу новых объектов - обязательно подтверждаем свои действия через коммит. А то вдруг мы дров наломаем - чтобы у базы была возможность наши действия проигнорировать.

In [ ]:
with Session() as session:
    session.add(Event_Log(name="test"))
    session.add(Event_Log(name="one more"))
    session.add(Event_Log(name="rioran"))
    session.commit()

Ну и наконец покажу мощь орм подхода, следите за руками - никакого SQL! Открываем сессию, проходимся по строкам таблицы, для каждой напечатаем душу события и дату-время. Обратите внимание, обращаюсь к дттм как к свойству класса!

Ну и в конце выведем количество строк в базе. Просто к таблице внутри нашей сессии дописываем агрегатную функцию!

In [ ]:
with Session() as session:
    for event in session.query(Event_Log).all():
        print(event, '// dttm =>', event.dttm)
    print("Строк в базе:", \
          session.query(Event_Log).count())

Та-да! Всё, хватит складывать данные вашего гениального приложения в текстовые файлы, вы готовы выдать обнимашку настоящей автоматизации, ведь теперь вы знаете, какое крутое в...